# Bookmark Data Analysis Notebook

This notebook demonstrates how to analyze the bookmark data collected by the deltaload tool.

In [ ]:
# Import necessary libraries
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timezone
from collections import Counter
import re

# Configure plots
plt.style.use('ggplot')
sns.set_palette('viridis')
%matplotlib inline

## Loading the Data

We'll load the bookmarks data from the JSONL file generated by deltaload.

In [ ]:
# Load the JSONL data
bookmarks_path = '/Users/imac/Desktop/ETL/deltaload/data-bookmark.jsonl'

# Function to parse JSONL file
def load_jsonl(filename):
    data = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"Error parsing line: {line[:100]}...")
    return data

# Load the data
bookmarks = load_jsonl(bookmarks_path)
print(f"Loaded {len(bookmarks)} bookmarks from {bookmarks_path}")

# Convert to DataFrame
df = pd.DataFrame(bookmarks)
df.head()

## Data Preprocessing

Now let's do some preprocessing on the data.

In [ ]:
# Convert dates to datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# Parse metadata
def parse_metadata(metadata_str):
    try:
        return json.loads(metadata_str)
    except (json.JSONDecodeError, TypeError):
        return {}

df['parsed_metadata'] = df['metadata'].apply(parse_metadata)

# Extract domain from URL
def extract_domain(url):
    if not isinstance(url, str):
        return 'unknown'
    
    match = re.search(r'https?://([\w.-]+)/', url)
    if match:
        return match.group(1)
    else:
        match = re.search(r'https?://([\w.-]+)', url)
        return match.group(1) if match else 'unknown'

df['domain'] = df['url'].apply(extract_domain)

# Display data types and missing values
print("Data Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())

# Basic statistics
df.describe(include='all')

## Data Analysis

### Source Distribution

In [ ]:
# Count by source
source_counts = df['source'].value_counts()
print("Bookmark Sources:")
print(source_counts)

# Plot source distribution
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=source_counts.index, y=source_counts.values)
plt.title('Distribution of Bookmark Sources')
plt.xlabel('Source')
plt.ylabel('Count')
plt.xticks(rotation=45)

# Add count labels on top of bars
for i, count in enumerate(source_counts.values):
    ax.text(i, count + 10, f'{count:,}', ha='center')

plt.tight_layout()
plt.show()

### Timeline Analysis

In [ ]:
# Group by date and source
df['date'] = df['created_at'].dt.date
date_source_counts = df.groupby(['date', 'source']).size().unstack().fillna(0)

# Plot timeline
plt.figure(figsize=(16, 8))
date_source_counts.plot(kind='line', ax=plt.gca())
plt.title('Bookmarks by Source Over Time')
plt.xlabel('Date')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)
plt.legend(title='Source')
plt.tight_layout()
plt.show()

# Monthly aggregation
df['year_month'] = df['created_at'].dt.to_period('M')
monthly_counts = df.groupby(['year_month', 'source']).size().unstack().fillna(0)

plt.figure(figsize=(16, 8))
monthly_counts.plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('Monthly Bookmarks by Source')
plt.xlabel('Month')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.legend(title='Source')
plt.tight_layout()
plt.show()

### Domain Analysis

In [ ]:
# Top domains
top_domains = df['domain'].value_counts().head(20)
print("Top 20 Domains:")
print(top_domains)

plt.figure(figsize=(12, 8))
ax = sns.barplot(x=top_domains.values, y=top_domains.index)
plt.title('Top 20 Domains in Bookmarks')
plt.xlabel('Count')
plt.ylabel('Domain')

# Add count labels
for i, count in enumerate(top_domains.values):
    ax.text(count + 1, i, f'{count:,}', va='center')

plt.tight_layout()
plt.show()

### Content Analysis

In [ ]:
# Text length analysis
df['content_length'] = df['content'].apply(lambda x: len(x) if isinstance(x, str) else 0)

plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='content_length', hue='source', bins=50, log_scale=(False, True))
plt.title('Distribution of Content Length by Source')
plt.xlabel('Content Length (characters)')
plt.ylabel('Count (log scale)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Average content length by source
avg_length_by_source = df.groupby('source')['content_length'].mean().sort_values(ascending=False)
print("Average Content Length by Source:")
print(avg_length_by_source)

plt.figure(figsize=(10, 6))
avg_length_by_source.plot(kind='bar')
plt.title('Average Content Length by Source')
plt.xlabel('Source')
plt.ylabel('Average Length (characters)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### GitHub-specific Analysis

In [ ]:
# Filter to GitHub data
github_df = df[df['source'] == 'github'].copy()

# Extract repository language and stars if available
def extract_language(metadata):
    if isinstance(metadata, dict):
        return metadata.get('language', 'Unknown')
    return 'Unknown'

def extract_stars(metadata):
    if isinstance(metadata, dict):
        return metadata.get('stars', 0)
    return 0

github_df['language'] = github_df['parsed_metadata'].apply(extract_language)
github_df['stars'] = github_df['parsed_metadata'].apply(extract_stars)

# Top languages
top_languages = github_df['language'].value_counts().head(15)
print("Top 15 Languages in GitHub Repositories:")
print(top_languages)

plt.figure(figsize=(12, 8))
ax = sns.barplot(x=top_languages.values, y=top_languages.index)
plt.title('Top 15 Languages in GitHub Repositories')
plt.xlabel('Count')
plt.ylabel('Language')

# Add count labels
for i, count in enumerate(top_languages.values):
    ax.text(count + 1, i, f'{count:,}', va='center')

plt.tight_layout()
plt.show()

# Stars distribution
plt.figure(figsize=(12, 6))
sns.histplot(data=github_df, x='stars', bins=50, log_scale=(False, True))
plt.title('Distribution of Stars in GitHub Repositories')
plt.xlabel('Stars Count (clipped at 95th percentile)')
plt.ylabel('Repository Count (log scale)')
plt.xlim(0, github_df['stars'].quantile(0.95))  # Clip at 95th percentile for better visibility
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Twitter-specific Analysis

In [ ]:
# Filter to Twitter data
twitter_df = df[df['source'].isin(['twitter', 'twitter_like'])].copy()

# Extract metrics like retweet_count, favorite_count if available
def extract_retweet_count(metadata):
    if isinstance(metadata, dict):
        return metadata.get('retweet_count', 0)
    return 0

def extract_favorite_count(metadata):
    if isinstance(metadata, dict):
        return metadata.get('favorite_count', 0)
    return 0

twitter_df['retweet_count'] = twitter_df['parsed_metadata'].apply(extract_retweet_count)
twitter_df['favorite_count'] = twitter_df['parsed_metadata'].apply(extract_favorite_count)

# Engagement metrics
print("Twitter Engagement Metrics:")
print(twitter_df[['retweet_count', 'favorite_count']].describe())

# Plot engagement distribution
plt.figure(figsize=(12, 10))

plt.subplot(2, 1, 1)
sns.histplot(data=twitter_df, x='retweet_count', bins=50, log_scale=(False, True))
plt.title('Distribution of Retweet Counts')
plt.xlabel('Retweet Count (clipped at 95th percentile)')
plt.ylabel('Tweet Count (log scale)')
plt.xlim(0, twitter_df['retweet_count'].quantile(0.95))  # Clip at 95th percentile for better visibility
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
sns.histplot(data=twitter_df, x='favorite_count', bins=50, log_scale=(False, True))
plt.title('Distribution of Favorite Counts')
plt.xlabel('Favorite Count (clipped at 95th percentile)')
plt.ylabel('Tweet Count (log scale)')
plt.xlim(0, twitter_df['favorite_count'].quantile(0.95))  # Clip at 95th percentile for better visibility
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tweets over time
twitter_df['hour'] = twitter_df['created_at'].dt.hour
twitter_df['day_of_week'] = twitter_df['created_at'].dt.day_name()

# Create a day of week order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Plot heatmap of tweet activity by hour and day
plt.figure(figsize=(12, 8))
tweet_heatmap = pd.crosstab(twitter_df['day_of_week'], twitter_df['hour'])
# Reorder the index
tweet_heatmap = tweet_heatmap.reindex(day_order)

sns.heatmap(tweet_heatmap, cmap='viridis', annot=True, fmt='d')
plt.title('Tweet Activity by Hour and Day of Week')
plt.xlabel('Hour of Day (UTC)')
plt.ylabel('Day of Week')
plt.tight_layout()
plt.show()

### Raindrop.io-specific Analysis

In [ ]:
# Filter to Raindrop data
raindrop_df = df[df['source'] == 'raindrop'].copy()

# Extract folder and tags
def extract_folder(metadata):
    if isinstance(metadata, dict):
        return metadata.get('folder', 'Unsorted')
    return 'Unsorted'

def extract_tags(metadata):
    if isinstance(metadata, dict):
        tags = metadata.get('tags', [])
        if isinstance(tags, list):
            return tags
    return []

raindrop_df['folder'] = raindrop_df['parsed_metadata'].apply(extract_folder)
raindrop_df['tags'] = raindrop_df['parsed_metadata'].apply(extract_tags)

# Top folders
folder_counts = raindrop_df['folder'].value_counts().head(15)
print("Top 15 Folders:")
print(folder_counts)

plt.figure(figsize=(12, 8))
ax = sns.barplot(x=folder_counts.values, y=folder_counts.index)
plt.title('Top 15 Folders in Raindrop.io Bookmarks')
plt.xlabel('Count')
plt.ylabel('Folder')

# Add count labels
for i, count in enumerate(folder_counts.values):
    ax.text(count + 1, i, f'{count:,}', va='center')

plt.tight_layout()
plt.show()

# Tags analysis (flatten the list of tags)
all_tags = [tag for tags_list in raindrop_df['tags'] for tag in tags_list if tags_list]
tag_counts = Counter(all_tags)
top_tags = pd.Series(tag_counts).sort_values(ascending=False).head(20)

print("\nTop 20 Tags:")
print(top_tags)

plt.figure(figsize=(12, 8))
ax = sns.barplot(x=top_tags.values, y=top_tags.index)
plt.title('Top 20 Tags in Raindrop.io Bookmarks')
plt.xlabel('Count')
plt.ylabel('Tag')

# Add count labels
for i, count in enumerate(top_tags.values):
    ax.text(count + 1, i, f'{count:,}', va='center')

plt.tight_layout()
plt.show()

## Content Topic Modeling

Let's try to identify common topics or themes in the bookmark content.

In [ ]:
# Extract text content
df['clean_content'] = df['content'].apply(lambda x: str(x) if pd.notna(x) else '')

# Filter out very short content
df_for_topics = df[df['clean_content'].str.len() > 10].copy()

# Simple keyword extraction and frequency analysis
def extract_keywords(text):
    # Convert to lowercase and remove special characters
    text = re.sub(r'[^\w\s]', ' ', text.lower())
    
    # Split into words
    words = text.split()
    
    # Filter out common stop words (a very basic list)
    stop_words = set(['the', 'and', 'is', 'in', 'to', 'of', 'a', 'for', 'on', 'with', 'as', 'by', 'an', 'that', 'this', 'are', 'from'])
    words = [word for word in words if word not in stop_words and len(word) > 2]
    
    return words

# Apply to all content
all_keywords = []
for content in df_for_topics['clean_content']:
    all_keywords.extend(extract_keywords(content))

# Count keywords
keyword_counts = Counter(all_keywords)
top_keywords = pd.Series(keyword_counts).sort_values(ascending=False).head(30)

print("Top 30 Keywords:")
print(top_keywords)

plt.figure(figsize=(12, 10))
ax = sns.barplot(x=top_keywords.values, y=top_keywords.index)
plt.title('Top 30 Keywords in Bookmark Content')
plt.xlabel('Frequency')
plt.ylabel('Keyword')

# Add count labels
for i, count in enumerate(top_keywords.values):
    ax.text(count + 10, i, f'{count:,}', va='center')

plt.tight_layout()
plt.show()

## Recommendations Based on Analysis

Let's generate some simple recommendations based on the analysis.

In [ ]:
# Find most popular repositories by stars
top_repos = github_df.sort_values('stars', ascending=False).head(10)[['content', 'stars', 'url']]
print("Top 10 GitHub Repositories by Stars:")
print(top_repos)

# Find most engaging tweets
twitter_df['engagement'] = twitter_df['retweet_count'] + twitter_df['favorite_count']
top_tweets = twitter_df.sort_values('engagement', ascending=False).head(10)[['content', 'retweet_count', 'favorite_count', 'url']]
print("\nTop 10 Tweets by Engagement:")
print(top_tweets)

# Find most popular domains
print("\nTop 10 Domains:")
print(df['domain'].value_counts().head(10))

# Find most used programming languages
if 'language' in github_df.columns:
    print("\nTop Programming Languages:")
    print(github_df['language'].value_counts().head(10))

# Find most active weeks
df['week'] = df['created_at'].dt.isocalendar().week
df['year'] = df['created_at'].dt.isocalendar().year
df['year_week'] = df['year'].astype(str) + '-W' + df['week'].astype(str).str.zfill(2)
weekly_counts = df.groupby('year_week').size().sort_values(ascending=False).head(5)
print("\nMost Active Weeks:")
print(weekly_counts)